### Thai - Registry : Scraping translated data

### https://porta.fda.moph.go.th/FDA_SEARCH_ALL/MAIN/SEARCH_CENTER_MAIN.aspx

In [1]:
#!pip install plyer
#from plyer import notification
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import geckodriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from datetime import datetime as date
import pandas as pd, numpy as np
import warnings as ws
ws.filterwarnings("ignore")
import re, os, time, glob
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pickle
from typing import List
# Create directory if it doesn't exist
directory = "Fetched Data"
if not os.path.exists(directory):
    os.makedirs(directory)
# Initialize Firefox with options
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
geckodriver_autoinstaller.install()


def clean_str(string):
    string = string.replace('\xa0', " ").strip()
    string = string.replace('\n', "")
    string = string.replace('\t', "")
    return string.strip()

Firefox version:  122.0
Available geckodriver version:  v0.34.0


In [3]:
## Use IVD Gateway Major player to search MDA using company names
major_players = pd.read_csv("IVD_gateway_major_players_actual.csv")
major_players 
players = major_players.player_name.values.tolist()
## Split out company names delimited by '-'
keywords = []
for name in players:
    if '-' in name:
        spread_names = name.split('-')
        for i in spread_names:
            keywords.append(i.strip())
    else:
        keywords.append(name)

In [4]:
seen = set()
keywords_indexed = [x for x in keywords if not (x in seen or seen.add(x))]
len(keywords_indexed)

345

In [5]:
driver = webdriver.Firefox(options=options)
driver.get('https://addons.mozilla.org/en-US/firefox/addon/traduzir-paginas-web/?utm_source=addons.mozilla.org&utm_medium=referral&utm_content=search')

In [24]:
catch = dict()

In [32]:
catch['19 Antigen Rapid Test'] = []

In [34]:
print("Searching FDA TH using keywords...")

for keyword in keywords_indexed[270:]:
    collector = []
    if keyword not in catch.keys() and len(keyword) > 3:
        print(keyword)
        try:
            driver, total_results = load_search(keyword, driver)
            page = 0
            if total_results == 'No records':
                print(f"No results for {keyword}.")
            elif total_results == 'One Page':
                flag, table = extract_page_data(driver)
                collector.append(table)
            else:
                flag, table = extract_page_data(driver)
                collector.append(table)
                ## find next button until
                total_res = total_results.strip().split(" ")[0]
                page_count = round(int(total_res) / 50)
                while page <= page_count:
                    page += 1
                    wait = WebDriverWait(driver, 10)
                    next_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'rgPageNext')))
                    next_btn.click()
                    table = extract_page_data(driver)
                    collector.append(table)
                    time.sleep(1)
        except Exception as e:
            print(f"Error for {keyword}")
            raise(e)

       
    catch[keyword] = collector
    
    

Searching FDA TH using keywords...
Access Bio


In [36]:
len(catch.keys())

75

In [37]:
export = []
for keys in catch.keys():
    collector = catch[keys]
    print(len(collector))
    if collector:
        print(True)
        print(type(collector))
        for collect in collector:
            #print("Children")
            #print(type(collect))
            
            if isinstance(collect, tuple):
                #print(collect)
                df = collect[1]
                df.columns = ['Product type', 'CAT_NO', 'Certificate/License',
       'Product name Thai - English', 'Trade name', 'Licensee name',
       'Product details', 'Newcode', 'status', 'Unnamed: 9']
                export.append(df)
            else:
                collect.columns = ['Product type', 'CAT_NO', 'Certificate/License',
       'Product name Thai - English', 'Trade name', 'Licensee name',
       'Product details', 'Newcode', 'status', 'Unnamed: 9']
                export.append(collect)
                
              
print(len(export))


clean_export = []
for df in export:
    print(df.shape)
    
    df = df.dropna(axis=0, how='all')

    # Create a boolean mask where True indicates rows where the column contains 'page size'
    mask = df['Product type'].str.contains('Page size:', na=False)
    df = df[~mask]
    mask = df['CAT_NO'].str.contains('select',na=False)
    df = df[~mask]
 
    clean_export.append(df)
    
extract = pd.concat(clean_export)
extract

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
True
<class 'list'>
3
(57, 10)
(26, 10)
(26, 10)


,Product type,CAT_NO,Certificate/License,Product name Thai - English,Trade name,Licensee name,Product details,Newcode,status,Unnamed: 9
3,เครื่องมือแพทย์,7220397,CHE5903567,T-Cone Toric Platform T-Cone Toric Platform Ac...,Accessories biometer LENSTAR LS 900,บริษัท ป.เคมีอุปกรณ์จำกัด,T-Cone Toric Platform,U1MC000102012600000376376C,หมดอายุ,NaN
4,เครื่องมือแพทย์,7220384,CHE5903567,T-Cone (without Toric Planner) T-Cone (without...,Accessories biometer LENSTAR LS 901,บริษัท ป.เคมีอุปกรณ์จำกัด,T-Cone (without Toric Planner),U1MC000102012600000377377C,หมดอายุ,NaN
5,เครื่องมือแพทย์,7220396,CHE5903567,EyeSuite IOL Toric Planner EyeSuite IOL Toric ...,Accessories biometer LENSTAR LS 902,บริษัท ป.เคมีอุปกรณ์จำกัด,EyeSuite IOL Toric Planner,U1MC000102012600000378378C,หมดอายุ,NaN
6,เครื่องมือแพทย์,7220398,CHE5903567,Upgrade Kit Lenstar with white light illuminat...,Accessories biometer LENSTAR LS 903,บริษัท ป.เคมีอุปกรณ์จำกัด,Upgrade Kit Lenstar with white light illuminat...,U1MC000102012600000379379C,หมดอายุ,NaN
7,เครื่องมือแพทย์,7220399,CHE5903567,Upgrade Kit Lenstar with white light illuminat...,Accessories biometer LENSTAR LS 904,บริษัท ป.เคมีอุปกรณ์จำกัด,Upgrade Kit Lenstar with white light illuminat...,U1MC000102012600000380380C,หมดอายุ,NaN
...,...,...,...,...,...,...,...,...,...,...
17,medical equipment,H45561FC,65-2-2-2-0011393,Voluson Expert M5Sc-D Biopsy Kit,M5Sc-D Biopsy Kit,"GE Medical Systems (Thailand) Co., Ltd.",Accessory- Biopsy kit/ Biopsy guide,U1MN0001026562626700014025C,remain,NaN
18,medical equipment,H46701AD,65-2-2-2-0011393,Voluson Expert RSP Biopsy Starter Kit - CIVCO ...,RSP Biopsy Starter Kit - CIVCO Part # 742-327,"GE Medical Systems (Thailand) Co., Ltd.",Accessory- Biopsy kit/ Biopsy guide,U1MN0001026562626700014126C,remain,NaN
19,medical equipment,H46721R,65-2-2-2-0011393,Voluson Expert RIC Biopsy PEC 63 - Reusable CIVCO,RIC Biopsy PEC 63 - Reusable CIVCO,"GE Medical Systems (Thailand) Co., Ltd.",Accessory- Biopsy kit/ Biopsy guide,U1MN0001026562626700014227C,remain,NaN
20,medical equipment,H48681GF,65-2-2-2-0011393,Voluson Expert RIC Biopsy Starter Kit - Dispos...,RIC Biopsy Starter Kit - Disposable CIVCO,"GE Medical Systems (Thailand) Co., Ltd.",Accessory- Biopsy kit/ Biopsy guide,U1MN0001026562626700014328C,remain,NaN


In [38]:
extract.to_excel("TH FDA 300.xlsx")

In [22]:
def load_search(keyword, driver):
    
    driver.get('https://porta.fda.moph.go.th/FDA_SEARCH_ALL/MAIN/SEARCH_CENTER_MAIN.aspx')
    driver.maximize_window()
    time.sleep(2)
    try:
        pop_close = driver.find_element(By.XPATH,'//*[@id="myModal2"]/div/div/div[3]/button')
        pop_close.click()
    except Exception as e:
        print(e, keyword)
    #driver.find_element(By.XPATH , "//div[@class = 'modal-content']/div[@class='modal-footer']/button[@class = 'btn btn-default']").click()
    time.sleep(4)

    driver.find_element(By.XPATH , "//div[@class = 'col-lg-2 col-md-2']/table/tbody/tr/td/input[@id = 'ContentPlaceHolder1_R_LIST']").click()
    time.sleep(4)

    driver.find_element(By.ID , 'ContentPlaceHolder1_CheckBoxList1').find_element(By.XPATH , "//tbody/tr[7]").click()
    time.sleep(4)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    wait = WebDriverWait(driver, 10)
    search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_btn_search"]')))
    input_box = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_txt_search"]')
    input_box.send_keys(keyword)
    driver.execute_script("arguments[0].click();", search_btn)
    time.sleep(2)
    wait = WebDriverWait(driver, 10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    flag, table = extract_page_data(driver)
    if flag == 'No results':
        return driver, 'No records'
    elif flag == 'One Page':
        return driver, 'One Page'
    else:
        totals = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_RadGrid1_ctl00_ctl03_ctl01_PageSizeComboBox_Input"]')))
        totals.click()
        fifty = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_RadGrid1_ctl00_ctl03_ctl01_PageSizeComboBox_DropDown"]/div/ul/li[3]')
        fifty.click()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        flag, table = extract_page_data(driver)
        if flag == 'One Page':
            return driver, 'One Page'
        else:
            total_res_count = driver.find_element(By.CLASS_NAME,'rgInfoPart')
            total_res_count = total_res_count.text

            return driver, total_res_count
    
def extract_page_data(driver):
    time.sleep(1)
    wait = WebDriverWait(driver, 30)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source)
    data = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'rgMasterTable')))
    table = soup.find('table',{'class':'rgMasterTable'})
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    if 'No records to display.' in table.text:
        return 'No results', table
    else:
        table = pd.read_html(str(table))[0]
        try:
            total_res_count = driver.find_element(By.CLASS_NAME,'rgInfoPart')
            return 'Found', table   
        except NoSuchElementException:
            return 'One Page', table
        
        
        

In [33]:
extract_page_data(driver)

('One Page',
       ประเภทผลิตภัณฑ์        CAT_NO ใบสำคัญ/ใบอนุญาต  \
 0     เครื่องมือแพทย์  40230;005702       CHE5602513   
 1     เครื่องมือแพทย์  40240;005712       CHE5602513   
 2     เครื่องมือแพทย์  45560;005400       CHE5602513   
 3     เครื่องมือแพทย์  45840;009948       CHE5602513   
 4     เครื่องมือแพทย์  45950;009930       CHE5602513   
 5     เครื่องมือแพทย์  60300;110082       CHE5602513   
 6     เครื่องมือแพทย์  60400;110074       CHE5602513   
 7   medical equipment  61470;110075       CHE5602513   
 8   medical equipment  61480;110076       CHE5602513   
 9   medical equipment  62380;110072       CHE5602513   
 10  medical equipment  63390;110073       CHE5602513   
 11  medical equipment  64370;110071       CHE5602513   
 12  medical equipment  65570;110077       CHE5602513   
 13  medical equipment        005702       CHE6101357   
 14  medical equipment        005712       CHE6101357   
 15  medical equipment        005400       CHE6101357   
 16  medical equip

In [87]:
driver.quit()